# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, behavior

[2023-01-09 17:26:31,458][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-01-09 17:26:31,517][INFO]: Connected tobiasr@172.26.128.53:3306


Loading DLC 2.2.1.1...


/Users/trose/opt/miniconda3/envs/datajoint/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.conn()

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [17]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

Root: ['/datajoint-data/data/tobiasr', '/Users/trose/Data/']
Full: /Users/trose/Data/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat


### Initial check of tables

In [9]:
from adamacs.pipeline import session, event, trial

session.Session.delete()
print('Sessions:', len(session.Session()))
print('Trials  :', len(trial.Trial()))
print('Events  :', len(event.Event()))

[2023-01-09 17:30:29,393][INFO]: Deleting 649715 rows from `tobiasr_event`.`_event`
[2023-01-09 17:30:29,510][INFO]: Deleting 1 rows from `tobiasr_event`.`behavior_recording__file`
[2023-01-09 17:30:29,621][INFO]: Deleting 1 rows from `tobiasr_event`.`behavior_recording`
[2023-01-09 17:30:29,659][INFO]: Deleting 1 rows from `tobiasr_scan`.`_scan_info__field`
[2023-01-09 17:30:29,681][INFO]: Deleting 1 rows from `tobiasr_scan`.`_scan_info__scan_file`
[2023-01-09 17:30:29,693][INFO]: Deleting 1 rows from `tobiasr_scan`.`_scan_info`
[2023-01-09 17:30:29,713][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan_location`
[2023-01-09 17:30:29,733][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan_path`
[2023-01-09 17:30:29,744][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan`
[2023-01-09 17:30:29,763][INFO]: Deleting 1 rows from `tobiasr_session`.`project_session`
[2023-01-09 17:30:29,781][INFO]: Deleting 1 rows from `tobiasr_session`.`session_directory`
[2023-01-09 17:30:29,803][INFO]: Deletin

Deletes cancelled
Sessions: 1
Trials  : 0
Events  : 649715


## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [16]:
bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_object = Bpodfile(bpod_path)
bpod_object.ingest()

TypeError: can only concatenate str (not "int") to str

Check that insertion worked:

In [ ]:
trial.TrialEvent & 'trial_id=1'

We can also interact with bpod objects. For example:

In [ ]:
bpod_object.trial(1).attributes

In [ ]:
bpod_object.trial(1).events

# Add Harp recording

In [ ]:
from adamacs.pipeline import behavior, event
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

In [ ]:
behavior.HarpRecording.populate()

In [ ]:
behavior.HarpRecording.Channel()